In [2]:
!pip install pandas boto3
import pandas as pd
import boto3
import botocore
from io import StringIO

import zipfile




In [4]:
# Create an S3 client

bucket_name = 'sdc-security'
crimes_path = 'datasets/crimes-cams.csv'
density_path = 'datasets/densidad_poblacion_2020.csv'

local_crimes_path = "D:\Archivos\Social Data Challenge\QGIS\crimes-cams.csv"
local_density_path = "D:\Archivos\Social Data Challenge\data\densidad_poblacion_2020.csv"


In [5]:
def s3_bucket_csv_to_pd(bucket_name, file_path):
    
    try:
        # Create an S3 client
        s3 = boto3.client('s3')

        # Download the CSV file from S3
        s3_object = s3.get_object(Bucket=bucket_name, Key=file_path)

        # Read the CSV content
        csv_content = s3_object['Body'].read().decode('utf-8')

        # Create a DataFrame from the CSV content
        df = pd.read_csv(StringIO(csv_content))

        return df
    except botocore.exceptions.NoCredentialsError:
        # Handle the case where AWS credentials are missing or invalid
        print("AWS credentials are missing or invalid.")
        return None
    except botocore.exceptions.ClientError as e:
        # Handle specific S3 client errors
        if e.response['Error']['Code'] == 'NoSuchKey':
            # Handle the case where the S3 file does not exist
            print("S3 file does not exist.")
            return None
        else:
            # Handle other S3 client errors
            print("Error reading from S3:", e)
            return None
    except Exception as e:
        # Handle other exceptions
        print("Error:", e)
        return None


In [6]:
def load_dataframe(bucket, s3_file_path, local_file_path):
    df = s3_bucket_csv_to_pd(bucket, s3_file_path)
    
    if df is None:
        print(f"Reading DataFrame from local file: {local_file_path}")
        df = pd.read_csv(local_file_path)
    return df

In [7]:
gsi_crimes = load_dataframe(bucket_name, crimes_path, local_crimes_path)
density    = load_dataframe(bucket_name, density_path, local_density_path)
 

Error reading from S3: An error occurred (SignatureDoesNotMatch) when calling the GetObject operation: The request signature we calculated does not match the signature you provided. Check your key and signing method.
Reading DataFrame from local file: D:\Archivos\Social Data Challenge\QGIS\crimes-cams.csv
Error reading from S3: An error occurred (SignatureDoesNotMatch) when calling the GetObject operation: The request signature we calculated does not match the signature you provided. Check your key and signing method.
Reading DataFrame from local file: D:\Archivos\Social Data Challenge\data\densidad_poblacion_2020.csv


In [8]:
density.rename(columns = lambda x : x.lower() , inplace = True)
density = density[density.alcaldia == 'CUAUHTEMOC'] 

In [9]:
catalogo_path = "\datasets\catalogo_colonias.csv.zip"
repo_path ='D:\Archivos\Social Data Challenge\sdc-security'
zf = zipfile.ZipFile(repo_path+catalogo_path) 
catalogo= pd.read_csv(zf.open('catalogo_colonias.csv'))
print(catalogo.shape)

(1543, 6)


In [10]:
catalogo_cuau = catalogo[catalogo.alc == 'CUAUHTEMOC']

In [11]:
catalogo_cuau.colonia.unique()

array(['ALGARIN', 'AMPL. ASTURIAS', 'ASTURIAS', 'ATLAMPA', 'BUENAVISTA',
       'BUENOS AIRES', 'CENTRO', 'CONDESA', 'CUAUHTEMOC', 'DOCTORES',
       'ESPERANZA', 'EX HIPODROMO DE PERALVILLO', 'FELIPE PESCADOR',
       'GUERRERO', 'HIPODROMO', 'HIPODROMO CONDESA', 'JUAREZ', 'MAZA',
       'MORELOS', 'NONOALCO TLATELOLCO', 'OBRERA', 'PAULINO NAVARRO',
       'PERALVILLO', 'ROMA NORTE', 'ROMA SUR', 'SAN RAFAEL',
       'SAN SIMON TOLNAHUAC', 'SANTA MARIA INSURGENTES',
       'SANTA MARIA LA RIBERA', 'TABACALERA', 'TRANSITO', 'VALLE GOMEZ',
       'VISTA ALEGRE'], dtype=object)

In [12]:
homologation_dict = {
    'AMPL. ASTURIAS':['ASTURIAS (AMPL)'], 
    'BUENAVISTA':['BUENAVISTA I', 'BUENAVISTA II'], 
    'CENTRO':['CENTRO I','CENTRO II', 
              'CENTRO III', 'CENTRO IV', 
              'CENTRO V', 'CENTRO VI',
              'CENTRO VII', 'CENTRO VIII'],
    'DOCTORES':['DOCTORES I', 'DOCTORES II', 
                'DOCTORES III', 'DOCTORES IV', 'DOCTORES V'],
    'GUERRERO':['GUERRERO I', 'GUERRERO II', 
                'GUERRERO III', 'GUERRERO IV'],
    'HIPODROMO':['HIPODROMO I', 'HIPODROMO II'], 
    'MORELOS':['MORELOS I', 'MORELOS II', 'MORELOS III'],
    'NONOALCO TLATELOLCO':['NONOALCO-TLATELOLCO (U HAB) I', 
                           'NONOALCO-TLATELOLCO (U HAB) II',
                           'NONOALCO-TLATELOLCO (U HAB) III',
                           'NONOALCO-TLATELOLCO'], 
    'OBRERA': ['OBRERA I', 'OBRERA II','OBRERA III', 'OBRERA IV'], 
    'PERALVILLO': ['PERALVILLO I', 'PERALVILLO II'], 
    'ROMA NORTE': ['ROMA NORTE I', 'ROMA NORTE II', 'ROMA NORTE III'],
    'ROMA SUR': ['ROMA SUR I', 'ROMA SUR II'], 
    'SAN RAFAEL':['SAN RAFAEL I', 'SAN RAFAEL II'],
    'SANTA MARIA LA RIBERA':['SANTA MARIA LA RIBERA I', 
                             'SANTA MARIA LA RIBERA II',
                             'SANTA MARIA LA RIBERA IV'], 
    'SANTA MARIA INSURGENTES':['SANTA MARIA (U HAB)'],
    'ATLAMPA':[],
    'ALGARIN' :[], 
    'ASTURIAS':[], 
    'BUENOS AIRES':[], 
    'CONDESA':[], 
    'CUAUHTEMOC':[], 
    'ESPERANZA':[], 
    'EX HIPODROMO DE PERALVILLO':[], 
    'FELIPE PESCADOR':[],
    'HIPODROMO CONDESA':[], 
    'JUAREZ':[],
    'MAZA':[], 
    'PAULINO NAVARRO':[], 
    'SAN SIMON TOLNAHUAC':[], 
    'TABACALERA':[], 
    'TRANSITO':[],
    'VALLE GOMEZ':[], 
    'VISTA ALEGRE':[], 
}


In [13]:

# Create 'colonia_new' column by mapping values from the dictionary
density['colonia_new'] = density['colonia'].apply(
    lambda x: next((k for k, v in homologation_dict.items() if x in v), x))

sum_aggregation_columns = ['area_ha','pobtot2010', 'pobtot2020', 
                           'vivhab2010', 'vivhab2020']

for col in sum_aggregation_columns:
    # Create aggregated columns by sum
    density[f'{col}_new'] = density.groupby('colonia_new')[col].transform('sum')
    
    # Create columns related to density
    if col not in ['area_ha']:
        #{col[0:3]} gets 'pob' or 'viv'
        #{col[-4:]} gets '2010' or '2020'
        density[f'den{col[0:3]}{col[-4:]}'] = density[f'{col}_new'] / density['area_ha_new']




In [14]:
other_cols_to_drop = ['fid_1','cve_ent','alcaldia','cve_col','denpob10','denpob20','denviv10','denviv20','difdenpob','difdenviv','tcma10_20']

density.drop(columns=['colonia'] + sum_aggregation_columns + other_cols_to_drop, 
             inplace=True)


In [15]:
density.drop_duplicates(subset=['colonia_new'], inplace=True)
density.columns = density.columns.to_series().apply(lambda x: x.replace('_new', ''))
density.columns = density.columns.to_series().apply(lambda x: x.replace('tot', ''))
density[density.colonia =='DOCTORES']

,colonia,area_ha,pob2010,denpob2010,pob2020,denpob2020,vivhab2010,denviv2010,vivhab2020,denviv2020
446,DOCTORES,250.152324,44703,178.703117,49787,199.026734,14477,57.872738,17616,70.421093


In [16]:
density['dif_denpob'] = density['denpob2020'] - density['denpob2010'] 

In [17]:
density = density[['colonia','pob2010', 'pob2020', 'denpob2010','denpob2020', 'dif_denpob',
                  'area_ha','vivhab2010', 'vivhab2020','denviv2010','denviv2020']]


In [18]:
density.shape

(33, 11)

In [19]:
print(gsi_crimes.isnull().sum())
crimes_cams_density = pd.merge(gsi_crimes,density, on = ['colonia'], how = 'left')
print(crimes_cams_density.isnull().sum())
crimes_cams_density.head()

idcarpeta               0
delito                  0
categoria               0
alcaldia                0
colonia                 0
sexo_fem                0
edad                    0
anio_hecho              0
colonia_al              0
crimen_lat              0
crimen_lon              0
hecho_time              0
denuncia_t              0
cams_in_crime_20m       0
cams_in_crime_50m       0
cams_in_crime_100m      0
cams_in_crime_250m      0
cams_in_crime_500m      0
crimes_metro_20m        0
metro_in_crimes_50m     0
metro_in_crimes_100m    0
metro_in_crimes_250m    0
metro_in_crimes_500m    0
vial_20                 0
vial_50                 0
vial_100                0
vial_250                0
vial_500                0
dtype: int64
idcarpeta               0
delito                  0
categoria               0
alcaldia                0
colonia                 0
sexo_fem                0
edad                    0
anio_hecho              0
colonia_al              0
crimen_lat              0

,idcarpeta,delito,categoria,alcaldia,colonia,sexo_fem,edad,anio_hecho,colonia_al,crimen_lat,...,pob2010,pob2020,denpob2010,denpob2020,dif_denpob,area_ha,vivhab2010,vivhab2020,denviv2010,denviv2020
0,8324479,ROBO A TRANSEUNTE A BORDO DE TAXI PUBLICO Y PR...,DELITO DE BAJO IMPACTO,CUAUHTEMOC,DOCTORES,0,39,2018,DOCTORES I,19.42244,...,44703,49787,178.703117,199.026734,20.323617,250.152324,14477,17616,57.872738,70.421093
1,8324482,ROBO DE OBJETOS,DELITO DE BAJO IMPACTO,CUAUHTEMOC,MORELOS,0,40,2018,MORELOS II,19.44962,...,34684,35005,317.403934,320.341504,2.937570,109.274008,10042,10711,91.897425,98.019650
2,8324485,ROBO DE OBJETOS,DELITO DE BAJO IMPACTO,CUAUHTEMOC,ROMA NORTE,0,37,2018,ROMA NORTE III,19.41359,...,27770,25611,119.973900,110.646437,-9.327463,231.467010,10776,11296,46.555231,48.801771
3,8324496,ROBO A CASA HABITACION SIN VIOLENCIA,DELITO DE BAJO IMPACTO,CUAUHTEMOC,JUAREZ,0,36,2019,JUAREZ,19.42676,...,10184,8542,65.050659,54.562326,-10.488333,156.554908,4045,4130,25.837580,26.380521
4,8324527,ROBO A PASAJERO A BORDO DE METRO SIN VIOLENCIA,ROBO A PASAJERO A BORDO DEL METRO CON Y SIN VI...,CUAUHTEMOC,GUERRERO,0,32,2018,GUERRERO III,19.43769,...,42339,47116,276.852785,308.089370,31.236585,152.929652,13013,15631,85.091412,102.210394


In [20]:
crimes_cams_density['crimes_in_col'] = crimes_cams_density.groupby('colonia')['colonia'].transform('count')
crimes_cams_density['den_crime'] = crimes_cams_density['crimes_in_col'] / crimes_cams_density['area_ha']



In [21]:
crimes_cams_density.head()

,idcarpeta,delito,categoria,alcaldia,colonia,sexo_fem,edad,anio_hecho,colonia_al,crimen_lat,...,denpob2010,denpob2020,dif_denpob,area_ha,vivhab2010,vivhab2020,denviv2010,denviv2020,crimes_in_col,den_crime
0,8324479,ROBO A TRANSEUNTE A BORDO DE TAXI PUBLICO Y PR...,DELITO DE BAJO IMPACTO,CUAUHTEMOC,DOCTORES,0,39,2018,DOCTORES I,19.42244,...,178.703117,199.026734,20.323617,250.152324,14477,17616,57.872738,70.421093,4463,17.841130
1,8324482,ROBO DE OBJETOS,DELITO DE BAJO IMPACTO,CUAUHTEMOC,MORELOS,0,40,2018,MORELOS II,19.44962,...,317.403934,320.341504,2.937570,109.274008,10042,10711,91.897425,98.019650,1436,13.141277
2,8324485,ROBO DE OBJETOS,DELITO DE BAJO IMPACTO,CUAUHTEMOC,ROMA NORTE,0,37,2018,ROMA NORTE III,19.41359,...,119.973900,110.646437,-9.327463,231.467010,10776,11296,46.555231,48.801771,5917,25.563038
3,8324496,ROBO A CASA HABITACION SIN VIOLENCIA,DELITO DE BAJO IMPACTO,CUAUHTEMOC,JUAREZ,0,36,2019,JUAREZ,19.42676,...,65.050659,54.562326,-10.488333,156.554908,4045,4130,25.837580,26.380521,3921,25.045526
4,8324527,ROBO A PASAJERO A BORDO DE METRO SIN VIOLENCIA,ROBO A PASAJERO A BORDO DEL METRO CON Y SIN VI...,CUAUHTEMOC,GUERRERO,0,32,2018,GUERRERO III,19.43769,...,276.852785,308.089370,31.236585,152.929652,13013,15631,85.091412,102.210394,3172,20.741563


### Merging with total number of C5 cams on each colonia 

In [30]:
c5_cams_cuau = pd.read_csv('D:\Archivos\Social Data Challenge\QGIS\c5_cuau.csv')
c5_cams_cuau.colonia.unique()

array(['CENTRO', 'PERALVILLO', 'SAN SIMON TOLNAHUAC',
       'NONOALCO TLATELOLCO', 'GUERRERO', 'SANTA MARIA LA RIBERA',
       'ROMA SUR', 'VISTA ALEGRE', 'AMPL. ASTURIAS', 'HIPODROMO',
       'CONDESA', 'ATLAMPA', 'EX HIPODROMO DE PERALVILLO', 'BUENAVISTA',
       'SAN RAFAEL', 'HIPODROMO CONDESA', 'ALGARIN', 'OBRERA',
       'PAULINO NAVARRO', 'BUENOS AIRES', 'DOCTORES', 'ROMA NORTE',
       'TRANSITO', 'MORELOS', 'JUAREZ', 'SANTA MARIA INSURGENTES',
       'TABACALERA', 'ASTURIAS', 'ESPERANZA', 'CUAUHTEMOC', 'MAZA',
       'FELIPE PESCADOR', 'VALLE GOMEZ'], dtype=object)

In [31]:
catalogo_cuau.colonia.unique()

array(['ALGARIN', 'AMPL. ASTURIAS', 'ASTURIAS', 'ATLAMPA', 'BUENAVISTA',
       'BUENOS AIRES', 'CENTRO', 'CONDESA', 'CUAUHTEMOC', 'DOCTORES',
       'ESPERANZA', 'EX HIPODROMO DE PERALVILLO', 'FELIPE PESCADOR',
       'GUERRERO', 'HIPODROMO', 'HIPODROMO CONDESA', 'JUAREZ', 'MAZA',
       'MORELOS', 'NONOALCO TLATELOLCO', 'OBRERA', 'PAULINO NAVARRO',
       'PERALVILLO', 'ROMA NORTE', 'ROMA SUR', 'SAN RAFAEL',
       'SAN SIMON TOLNAHUAC', 'SANTA MARIA INSURGENTES',
       'SANTA MARIA LA RIBERA', 'TABACALERA', 'TRANSITO', 'VALLE GOMEZ',
       'VISTA ALEGRE'], dtype=object)

In [39]:
c5_agg = c5_cams_cuau.groupby('colonia').agg(count=('colonia','count'))
#c5_agg = consolidated.groupby('colonia').agg(count=('colonia','count'), alcaldia=('alcaldia', 'first'))
c5_agg.reset_index(inplace=True)
c5_agg.rename(columns= {'count':'cams_col_count'}, inplace = True)

crimes_cams_density=crimes_cams_density.merge(c5_agg, on =['colonia'], how = 'left')

In [45]:
crimes_cams_density['cams_density'] = crimes_cams_density['cams_col_count']/ crimes_cams_density['area_ha']
crimes_cams_density['crimes_cams_den'] = crimes_cams_density['crimes_in_col'] / crimes_cams_density['cams_col_count']
crimes_cams_density.head()


,idcarpeta,delito,categoria,alcaldia,colonia,sexo_fem,edad,anio_hecho,colonia_al,crimen_lat,...,area_ha,vivhab2010,vivhab2020,denviv2010,denviv2020,crimes_in_col,den_crime,cams_col_count,cams_density,crimes_cams_den
0,8324479,ROBO A TRANSEUNTE A BORDO DE TAXI PUBLICO Y PR...,DELITO DE BAJO IMPACTO,CUAUHTEMOC,DOCTORES,0,39,2018,DOCTORES I,19.42244,...,250.152324,14477,17616,57.872738,70.421093,4463,17.841130,93,0.371773,47.989247
1,8324482,ROBO DE OBJETOS,DELITO DE BAJO IMPACTO,CUAUHTEMOC,MORELOS,0,40,2018,MORELOS II,19.44962,...,109.274008,10042,10711,91.897425,98.019650,1436,13.141277,61,0.558230,23.540984
2,8324485,ROBO DE OBJETOS,DELITO DE BAJO IMPACTO,CUAUHTEMOC,ROMA NORTE,0,37,2018,ROMA NORTE III,19.41359,...,231.467010,10776,11296,46.555231,48.801771,5917,25.563038,110,0.475230,53.790909
3,8324496,ROBO A CASA HABITACION SIN VIOLENCIA,DELITO DE BAJO IMPACTO,CUAUHTEMOC,JUAREZ,0,36,2019,JUAREZ,19.42676,...,156.554908,4045,4130,25.837580,26.380521,3921,25.045526,67,0.427965,58.522388
4,8324527,ROBO A PASAJERO A BORDO DE METRO SIN VIOLENCIA,ROBO A PASAJERO A BORDO DEL METRO CON Y SIN VI...,CUAUHTEMOC,GUERRERO,0,32,2018,GUERRERO III,19.43769,...,152.929652,13013,15631,85.091412,102.210394,3172,20.741563,86,0.562350,36.883721


In [46]:
crimes_cams_density.to_csv('../datasets/crimes_cams_density.csv', index=False)
density.to_csv('../datasets/density_cuau.csv', index=False)